# Imports 

In [2]:
import pandas as pd
import numpy as np
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Reshaping 

In [ ]:
## simulated data from review session
np.random.seed(1129)
shop_names = ['Compass', 'Starbucks', 'Baked and Wired', 'Peets', 
              'Blue Bottle', 'Saxbys']
coffee_df = pd.concat([pd.DataFrame({'shop_name': shop_names,
                         'opening_time': np.random.choice(["8:00 AM", "9:00 AM", "10:00 AM"],
                                                       len(shop_names),
                                                       replace = True),
                         'closing_time': np.random.choice(["5:00 PM", "6:00 PM", 
                                                          "7:00 PM"],
                                                         len(shop_names),
                                                          replace = True),
                         'hourly_wage': np.random.uniform(14, 20,
                                                          len(shop_names)),
                        'year': 2019}),
                      pd.DataFrame({'shop_name': shop_names,
                         'opening_time': np.random.choice(["8:00 AM", "9:00 AM", "10:00 AM"],
                                                       len(shop_names),
                                                       replace = True),
                         'closing_time': np.random.choice(["3:00 PM", "4:00 PM",
                                                           "6:00 PM", 
                                                          "7:00 PM"],
                                                         len(shop_names),
                                                          replace = True),
                         'hourly_wage': np.random.uniform(14, 20,
                                                          len(shop_names)),
                        'year': 2021})]).sort_values(by = 'shop_name')
                      

coffee_df

## 1.1 Create a wide-format dataframe where we have the hourly wage for each year as a separate column

## 1.2 Create a long-format dataframe where we have the hourly wage for each year as a separate column

## 1.3 Practice for you

Using the original `coffee_df`:

- Reshape to wide format for opening_time and closing_time (you can remove hourly_wage) from the data
- Use list comprehension to subset the dataframe to the shop_name and 2019 times only (so shop_name, opening_time 2019 and closing time 2019)

# 2. Merging


## 2.1: Read in the two datasets 

- San Diego data: `naics_code` and `account_key`
- NAICS details data: `naics` 

Run code below; if pulling from github, pathname should be fine; if working elsewhere may need to edit path name at read in 

In [3]:
sd_df = pd.read_csv("../../public_data/sd_df.csv")
naics_df = pd.read_csv("../../public_data/naics_df.csv")

In [4]:
cols_sd_use = ["naics_code", "account_key"]
cols_naics_use = ["naics"]

sd_df[cols_sd_use] = sd_df[cols_sd_use].astype(str)
naics_df[cols_naics_use] = naics_df[cols_naics_use].astype(str)

sd_df.head()
naics_df.head()

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4


,naics,naics_description
0,111140,Wheat Farming
1,111160,Rice Farming
2,111150,Corn Farming
3,111110,Soybean Farming
4,111120,Oilseed (except Soybean) Farming


## 2.2 "Inner join"- retain only San Diego businesses with details on their NAICS code

- Use the `naics_code` column in the San Diego business data as the join key
- Use the `naics` column in the NAICS code details data as the join key

- Do an inner join of the San Diego data onto the NAICS code details using these join keys
- After the inner join, print some examples of San Diego businesses lost in the merge
- Use value_counts() on the `naics_nchar` column in the San Diego data to see why they might have gotten lost


## 2.3 "Left join"- retain all sd businesses even if naics code isn't in the naics_details df

- Using the same join keys as above, and treating the San Diego businesses as the left hand side data, left join the naics code details onto the San Diego businesses
- Use the `indicator` argument within merge to create an indicator, `naics_merge_status`, to help with later merge diagnostics and examine sample of ones that didn't merge]
- Use the `suffixes` argument within merge to add `_sd` as the left suffix, `_census` as the right suffix


## 2.4 Use group by and agg to see if there are differences in merge rates by area

- Using the left-joined dataframe created in previous step, create a boolean indicator---`is_lost` if the merge indicator is equal to "left_only"
- Group by `council_district` and use the shortcut of taking the mean of a True/False indicator to find the proportions in order to find the proportion lost in the merge (so in the left join, ones that failed to match to `naics_df`) by council_district


## 2.5 Practice for you: add lagging 0's and see if merge rate from left join improves

You noticed earlier that a big reason for non-matches is that the San Diego tax certificate NAICS codes were oftentimes not 6-digits long, while the Census ones were always 6 digits

You wonder if this is an issue where 0's in some of the SD's data naics codes got cutoff (eg 540000 became 54) and if adding these lagging zeros would improve the merge rate in a left join

- Using one of two approaches, pad the `naics_code` column in `sd_df` with 0's to get that column up to 6-digits: (1) str.pad in pandas (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html); (2) for more of a challenge, write your own function that checks the # of digits in the naics code string and pads with 0's at the end up to 6 digits and use row-wise apply---`df.apply(funcname, axis = 1)`---to execute it
- Perform the same left join as in 2.3 and see how the match rate changes
- Create an indicator variable--`is_new_match`---for new matches under the padded NAICS code; compare the `naics_description` column from San Diego versus Census in the new dataset for a sample of these new matches and comment on whether the padding seems to be correct